In [276]:
import pandas as pd
import numpy as np
import requests 
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer
import seaborn as sns



# 

# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse import hstack
# from matplotlib import pyplot as plt
# import eli5



In [172]:
lem = WordNetLemmatizer()
stemmer = LancasterStemmer()



In [10]:
movies = requests.get('https://www.whats-on-netflix.com/wp-content/plugins/whats-on-netflix/json/movie.json?_=1619550477680')
tv = requests.get('https://www.whats-on-netflix.com/wp-content/plugins/whats-on-netflix/json/tv.json?_=1619550201106')

movie_df = pd.read_json(movies.content)
tv_df = pd.read_json(tv.content)
content = pd.concat([movie_df,tv_df]).reset_index(drop=True)
content.head()

,title,type,titlereleased,image_landscape,image_portrait,rating,quality,actors,director,category,imdb,runtime,netflixid,date_released,description,language
0,#Alive,Movie,2020,https://www.whats-on-netflix.com/wp-content/up...,https://www.whats-on-netflix.com/wp-content/up...,TV-MA,SuperHD,"Yoo Ah-in, Park Shin-hye",Cho Il,"Horror, Zombie, Korean",6.2/10,98 mins,81240831,2020-09-08,"As a grisly virus rampages a city, a lone man ...",Korean
1,#AnneFrank - Parallel Stories,Movie,2019,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,TV-14,SuperHD,"Helen Mirren, Gengher Gatti","Sabina Fedeli, Anna Migotto",Documentary,,94 mins,81264660,2020-07-01,"Through her diary, Anne Frank's story is retol...",English
2,#cats_the_mewvie,Movie,2020,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,TV-14,SuperHD,,Michael Margolis,,5.2/10,89 minutes,81218137,2020-02-05,This pawesome documentary explores how our fel...,English
3,#FriendButMarried,Movie,2018,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,TV-G,SuperHD,"Adipati Dolken, Vanesha Prescilla, Refal Hady,...",Rako Prijanto,"Biography, Drama",7.1/10,102 min,81260630,2020-05-22,"Pining for his high school crush for years, a ...",Indonesian
4,#FriendButMarried 2,Movie,2020,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,TV-G,SuperHD,"Adipati Dolken, Mawar Eva de Jongh, Vonny Corn...",Rako Prijanto,"Biography, Drama, Romance",,104 mins,81260637,2020-06-28,As Ayu and Ditto finally transition from best ...,Indonesian


## Cleanup

In [294]:
# Turn columns with comma separated items into lists
# content['category'] = content['category'].apply(lambda x: x.split(","))
# content['actors'] = content['actors'].apply(lambda x: x.split(","))

# Clean up data for tf_idf
content["description_clean"] = content['description'].str.replace('[^\w\s]','').str.lower()

content["description_clean"] = content["description_clean"].apply(lambda x: x.split(" "))
content["description_clean"] = content["description_clean"].apply(lambda x: [item.replace("'",' ') for item in x])

stoplist = stopwords.words('english')
content["description_clean"] = content["description_clean"].apply(lambda x: [item for item in x if item not in stoplist])

content["description_clean"] = content["description_clean"].apply(lambda x: [item for item in x if item != ""])


lem = WordNetLemmatizer()
content["description_clean"] = content["description_clean"].apply(lambda x: [lem.lemmatize(item) for item in x])

lancaster=LancasterStemmer()
content["description_clean"] = content["description_clean"].apply(lambda x: [lancaster.stem(item) for item in x])


content['category'] = content['category'].apply(lambda x: [item.lower().strip() for item in x])

content.description_clean[0:5]


content.head()


,title,type,titlereleased,image_landscape,image_portrait,rating,quality,actors,director,category,imdb,runtime,netflixid,date_released,description,language,description_clean
0,#Alive,Movie,2020,https://www.whats-on-netflix.com/wp-content/up...,https://www.whats-on-netflix.com/wp-content/up...,TV-MA,SuperHD,"[Yoo Ah-in, Park Shin-hye]",Cho Il,"[horror, zombie, korean]",6.2/10,98 mins,81240831,2020-09-08,"As a grisly virus rampages a city, a lone man ...",Korean,"[gris, vir, ramp, city, lon, man, stay, lock, ..."
1,#AnneFrank - Parallel Stories,Movie,2019,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,TV-14,SuperHD,"[Helen Mirren, Gengher Gatti]","Sabina Fedeli, Anna Migotto",[documentary],,94 mins,81264660,2020-07-01,"Through her diary, Anne Frank's story is retol...",English,"[diary, an, frank, story, retold, alongsid, fi..."
2,#cats_the_mewvie,Movie,2020,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,TV-14,SuperHD,[],Michael Margolis,[],5.2/10,89 minutes,81218137,2020-02-05,This pawesome documentary explores how our fel...,English,"[pawesom, docu, expl, felin, friend, becam, on..."
3,#FriendButMarried,Movie,2018,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,TV-G,SuperHD,"[Adipati Dolken, Vanesha Prescilla, Refal Ha...",Rako Prijanto,"[biography, drama]",7.1/10,102 min,81260630,2020-05-22,"Pining for his high school crush for years, a ...",Indonesian,"[pin, high, school, crush, year, young, man, p..."
4,#FriendButMarried 2,Movie,2020,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,TV-G,SuperHD,"[Adipati Dolken, Mawar Eva de Jongh, Vonny C...",Rako Prijanto,"[biography, drama, romance]",,104 mins,81260637,2020-06-28,As Ayu and Ditto finally transition from best ...,Indonesian,"[ayu, ditto, fin, transit, best, friend, newly..."


In [ ]:
flat_list = [item for sublist in content["description_clean"] for item in sublist]
df = pd.DataFrame(flat_list, columns = ['cat'])
df.value_counts()[0:5]

## Modeling

### TF-IDF

In [197]:

# vectorizer = CountVectorizer()
# test = vectorizer.fit_transform(content.description_clean)
# test

tfIdfTransformer = TfidfTransformer(use_idf=True)
vectorizer = CountVectorizer(analyzer=lambda x: x)
wordcount = vectorizer.fit_transform(content.description_clean)
tfidf = tfIdfTransformer.fit_transform(wordcount)
dense = tfidf.todense()
denselist = dense.tolist()
freq_df = pd.DataFrame(denselist, columns=vectorizer.get_feature_names())
freq_df

,007,009,00s,1,10,100,1000,10000,100000,102yearold,...,zé,álex,álvaro,ángel,ömer,über,überelit,łukasz,ōara,şeref
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.29931,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### One vs rest modeling

In [299]:
# Remove rows where no categories are present
combo_df = content[content.category.astype(str).str.len() > 4]
combo_df = combo_df[['category']]
combo_df = combo_df.merge(freq_df,left_index=True, right_index=True)
combo_df_ovm = combo_df.copy()

# Evaluate model on just a single category to start
combo_df_ovm.category = combo_df_ovm.category.apply(lambda x: x[0])


In [298]:
# Test train split 
y = combo_df_ovm.pop('category')
X = combo_df_ovm
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
logit = LogisticRegression(C=1, solver='lbfgs', multi_class='multinomial', random_state=17)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
cv_results = cross_val_score(logit, X_train, y_train, cv=skf, scoring='f1_micro')
cv_results

/Users/dmitri.mirakyanopendoor.com/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


array([0.28457447, 0.30851064, 0.30053191])

In [304]:
logit.fit(X_train, y_train)
logit.score(X_test, y_test)

0.3367768595041322

In [305]:
onevsrest = OneVsRestClassifier(LinearSVC(random_state=0))
onevsrest.fit(X_train, y_train)

OneVsRestClassifier(estimator=LinearSVC(random_state=0))

In [306]:
onevsrest.score(X_test, y_test)

0.384297520661157

In [263]:
# Gradient boosting (untuned) did noticeable worse.
xgbrest = OneVsRestClassifier(GradientBoostingClassifier())
xgbrest.fit(X_train, y_train)
xgbrest.score(X_test, y_test)

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [ ]:
# What does setting the threshold actually do?
# Why does it run for infinity

onevsrest = OneVsRestClassifier(SVC(kernel='linear', probability=True, class_weight=None))
onevsrest.fit(X_train, y_train)
proba = clf.predict_proba(X)
# y_pred_prob = onevsrest.predict_proba(X_test)
# y_pred_label = (y_pred_prob >= t).astype(int)
t=0.3
y_pred_prob = onevsrest.predict_proba(X_test)
y_pred_label = (y_pred_prob >= t).astype(int)



# 1 vs rest.
# First approach - pick the first category.
# New setup - 1 row = 1 y. Multiple categories but multiple rows. 
# The model will predict the likelihood of being in any of the categories.

# Logistic regression is slightly different. 
# It will create a pair.
# It will then aggregate via a method. From all the pairwise models, what is the total likelihood of being.

# Alternative
# Multi-label category. 
# Only take the top 300 words in the feature space.
# Also only look for the most common categories in the outcome.
# Also probably understand how the model you're using work
# Example/reference: https://www.kaggle.com/questions-and-answers/66693

# Other measures: fake description



## Multi-label classication modeling

In [297]:
# Start by evaluating the quality of the category items

flat_list = [item for sublist in combo_df["category"] for item in sublist]
df = pd.DataFrame(flat_list, columns = ['cat'])
df.value_counts()[0:30]

cat        
drama          602
comedy         429
thriller       245
action         202
documentary    198
romance        192
animation      165
crime          149
family         147
adventure      116
fantasy        107
mystery         96
horror          88
sci-fi          84
biography       57
music           55
kids            50
reality-tv      34
sport           34
musical         34
history         34
war             33
short           21
romantic        18
animated        18
children        17
game-show       16
stand-up        14
nollywood       12
christmas       10
dtype: int64

In [307]:
combo_df_mlc = combo_df.copy()

y = combo_df_mlc.pop('category')
m = MultiLabelBinarizer()
y = m.fit_transform(y)
X = combo_df_mlc
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [308]:
# logit = LogisticRegression(C=1, solver='lbfgs', multi_class='multinomial', random_state=17)
# mlc = MultiOutputClassifier(logit)

forest = RandomForestClassifier(random_state=1)
mlc = MultiOutputClassifier(forest,n_jobs=-1)
mlc.fit(X_train, y_train)


MultiOutputClassifier(estimator=RandomForestClassifier(random_state=1),
                      n_jobs=-1)

In [310]:
mlc.score(X_test, y_test)

0.05991735537190083

In [311]:
output = mlc.predict(X_test)
o = m.inverse_transform(output)
o

[(),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 ('documentary',),
 ('documentary',),
 (),
 (),
 ('drama',),
 (),
 (),
 ('animation',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 ('comedy', 'drama'),
 ('comedy', 'drama'),
 (),
 (),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 ('drama',),
 ('drama',),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 (),
 ('drama',),
 (),
 ('drama',),
 (),
 (),
 ('drama',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('drama',),
 (),
 ('drama',),
 (),
 ('comedy',),
 (),
 (),
 ('drama',),
 ('documentary',),
 ('drama',),
 (),
 (),
 (),
 ('documentary',),
 (),
 ('drama',),
 (),
 ('drama',),
 ('drama',),
 (),
 ('drama',),
 (),

In [96]:
train, test = train_test_split(content, test_size=0.5)
train

,title,type,titlereleased,image_landscape,image_portrait,rating,quality,actors,director,category,imdb,runtime,netflixid,date_released,description,language,description_clean
3607,Whose Streets?,Movie,2017,https://www.whats-on-netflix.com/wp-content/up...,https://www.whats-on-netflix.com/wp-content/up...,R,SuperHD,"[Lezley McSpadden, Michael Brown Sr., David ...","Sabaah Folayan, Damon Davis(co-director)",[Documentary],None,102 min,80168085,2020-11-16,"Powered by activists and leaders, this documen...",English,"[powered, activist, leader, documentary, follo..."
2527,Salaakhen,Movie,1975,https://occ-0-114-116.1.nflxso.net/art/7e232/9...,https://occ-0-114-116.1.nflxso.net/art/7e232/9...,TV-14,SuperHD,"[Shashi Kapoor, Sulakshana Pandit, Mehmood, ...",A. Salaam,[],5.3/10,133 minutes,80158479,2017-04-01,Two close childhood friends take drastically d...,Hindi,"[two, close, childhood, friend, take, drastica..."
882,Echcharikkai,Movie,2018,https://occ-0-1722-1723.1.nflxso.net/art/8be71...,https://occ-0-1722-1723.1.nflxso.net/art/8be71...,TV-14,SuperHD,"[Sathyaraj, Varalakshmi Sarathkumar, Kishore...",Sarjun,[],6.5/10,126 minutes,81045065,2019-01-15,"After kidnapping a millionaire’s daughter, two...",Tamil,"[kidnapping, millionaire, daughter, two, men, ..."
1126,Girl,Movie,2018,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,R,SuperHD,"[Katelijne Damen, Oliver Bodart, Magali Elal...",Lukas Dhont,[],7.2/10,105 minutes,81004374,2019-03-15,Fifteen-year-old ballet dancer Lara faces phys...,French,"[fifteenyearold, ballet, dancer, lara, face, p..."
682,Cook Off,Movie,2017,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,https://occ-0-1722-1723.1.nflxso.net/dnm/api/v...,TV-G,SuperHD,"[Tendaiishe Chitima, Tomas Brickhill, Chirik...",Tomas Brickhill,"[Comedy, Romance]",6.4/10,101 min,81273196,2020-06-01,"Yearning for a better life, a single mother wi...",English,"[yearning, better, life, single, mother, passi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3184,The Pixar Story,Movie,2007,https://occ-0-1722-1723.1.nflxso.net/art/66734...,https://occ-0-1722-1723.1.nflxso.net/art/66734...,G,SuperHD,[Stacy Keach],Leslie Iwerks,[],7.8/10,88 minutes,70083532,2018-11-18,Go behind the scenes at Pixar Animation Studio...,English,"[go, behind, scene, pixar, animation, studio, ..."
3619,Wine Country,Movie,2019,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,https://occ-0-1723-1722.1.nflxso.net/dnm/api/v...,R,SuperHD,"[Amy Poehler, Maya Rudolph, Ana Gasteyer, R...",Amy Poehler,[],5.4/10,103 minutes,80194950,2019-05-10,When longtime friends meet up for a wine-soake...,English,"[longtime, friend, meet, winesoaked, birthday,..."
1724,Love and Shukla,Movie,2017,https://occ-0-114-116.1.nflxso.net/art/498c6/2...,https://occ-0-114-116.1.nflxso.net/art/498c6/2...,TV-MA,SuperHD,"[Saharsh Kumar Shukla, Taneea Rajawat, Hima ...",Jatla Siddartha,[],7.3/10,110 minutes,81013710,2018-09-01,A sexually inexperienced rickshaw driver strug...,Hindi,"[sexually, inexperienced, rickshaw, driver, st..."
1576,Khido Khundi,Movie,2018,https://occ-0-116-114.1.nflxso.net/art/effe0/d...,https://occ-0-116-114.1.nflxso.net/art/effe0/d...,TV-14,SuperHD,"[Ranjit Bawa, Mandy Takhar, Manav Vij, Elna...",Rohit Jugraj,[],6.5/10,150 minutes,81024696,2018-10-01,An embittered former hockey star must engage w...,Panjabi,"[embittered, former, hockey, star, must, engag..."


## Bit of exploration

In [81]:

# TV shows tragically don't list # of seasons or episodes

content.groupby(['language']).title.count().sort_values(ascending=False)[0:50]
# English is most common language by far
# 34/8000 have no language at all

content.loc[content.language.isnull()]
content.loc[content.language == '']
# Content without languages doesn't seem to follow any specific pattern

content.groupby(['rating']).title.count().sort_values(ascending=False)[0:50]
# 3.7k movies, 3.2 TV shows

content.groupby(['type']).title.count().sort_values(ascending=False)[0:50]
# 3.7k movies, 3.2 TV shows


# content.dtypes
# content[["title"]] = content[["title"]].astype(str)

content.groupby(['rating','language']).title.count().sort_values(ascending=False)[0:50]
content.groupby(['rating']).title.count().sort_values(ascending=False)[0:50]
# ~25% of movies are Mature (2.5k) followed by TV-14 (1523)


# content.head()


content_by_cat = content.explode('category').reset_index(drop=True)
content_by_cat.category[0:5].dtypes



dtype('O')